# Image Analysis

 - *Description and tag generation* - determining an appropriate caption for an image, and identifying relevant "tags" that can be used as keywords to indicate its subject.
 - *Object detection* - detecting the presence and location of specific objects within the image.
 - *People detection* - detecting the presence, location, and features of people in the image.
- *Background removal* - detecting the background in an image and output the image with the background transparent or a greyscale alpha matte image.
- *Optical character recognition* - reading text in the image.
- *Smart thumbnail generation* - identifying the main region of interest in the image to create a smaller "thumbnail" version.

In [ ]:
import os
import requests

from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
from PIL import Image, ImageDraw, ImageFont

load_dotenv(override=True)
endpoint = os.environ.get("AZURE_VISION_ENDPOINT")
api_key = os.environ.get("AZURE_VISION_API_KEY")
credential = AzureKeyCredential(api_key)

image_analysis_client = ImageAnalysisClient(endpoint, credential)

image_path = 'images/DSC08294.jpg'
with open(image_path, 'rb') as f:
    image_data = f.read()

image = Image.open(image_path)

image

In [ ]:
from azure.ai.vision.imageanalysis.models import VisualFeatures

image_analysis = image_analysis_client.analyze(
    image_data=image_data, visual_features=[VisualFeatures.CAPTION]
)

print(image_analysis)
image

In [ ]:
image_analysis = image_analysis_client.analyze(
    image_data=image_data, visual_features=[VisualFeatures.TAGS]
)
print(image_analysis)
image

In [ ]:
image_analysis = image_analysis_client.analyze(
    image_data=image_data, visual_features=[VisualFeatures.OBJECTS]
)
print(image_analysis)

image = Image.open(image_path)
draw = ImageDraw.Draw(image)
font = ImageFont.truetype("arial.ttf", 32)
color = "red"

for detected_object in image_analysis.objects.list:
    coords = detected_object.bounding_box
    tags = ", ".join([tag.name for tag in detected_object.tags])
    bounding_box = (
        (coords.x, coords.y),
        (coords.x + coords.width, coords.y + coords.height),
    )

    draw.rectangle(bounding_box, outline=color, width=3)
    draw.text((coords.x, coords.y), tags, font=font, fill=color)

image

In [ ]:
image_analysis = image_analysis_client.analyze(
    image_data=image_data, visual_features=[VisualFeatures.PEOPLE]
)
print(image_analysis)

image = Image.open(image_path)
draw = ImageDraw.Draw(image)
font = ImageFont.truetype("arial.ttf", 32)
color = "red"

for detected_person in image_analysis.people.list:
    coords = detected_person.bounding_box
   
    bounding_box = (
        (coords.x, coords.y),
        (coords.x + coords.width, coords.y + coords.height),
    )

    draw.rectangle(bounding_box, outline=color, width=3)


image

In [ ]:
image_analysis = image_analysis_client.analyze(
    image_data=image_data, visual_features=[VisualFeatures.DENSE_CAPTIONS]
)
print(image_analysis)

image = Image.open(image_path)
draw = ImageDraw.Draw(image)
font = ImageFont.truetype("arial.ttf", 32)
color = "red"

for detected_caption in image_analysis.dense_captions.list:
    coords = detected_caption.bounding_box
    caption = detected_caption.text

    bounding_box = (
        (coords.x, coords.y),
        (coords.x + coords.width, coords.y + coords.height),
    )
    draw.text((coords.x, coords.y), caption, font=font, fill=color)
    draw.rectangle(bounding_box, outline=color, width=3)


image

In [ ]:
image_analysis = image_analysis_client.analyze(
    image_data=image_data, visual_features=[VisualFeatures.SMART_CROPS]
)
print(image_analysis)

image = Image.open(image_path)

for smart_crop in image_analysis.smart_crops.list:
    coords = smart_crop.bounding_box
    bounding_box = ( # different format
        coords.x,
        coords.y,
        coords.x + coords.width,
        coords.y + coords.height,
    )
    cropped_image = image.crop(bounding_box)

cropped_image

In [ ]:
#switching to a different photo that has a background/foreground

image_path = 'images\DSC08037.jpg'
with open(image_path, 'rb') as f:
    image_data = f.read()

image = Image.open(image_path)
image

In [ ]:
image = Image.open(image_path)
draw = ImageDraw.Draw(image)

image_analysis = image_analysis_client.analyze(
    image_data=image_data, visual_features=[VisualFeatures.READ]
)
print(image_analysis)

for detected_text in image_analysis.read.blocks:
    for detected_line in detected_text.lines:
        text = detected_line.text
        polygon = detected_line.bounding_polygon

        points = [(point['x'], point['y']) for point in polygon]
        draw.polygon(points, outline='red')
        draw.text((points[0][0], points[0][1]), text, fill='blue')

image

In [ ]:
#switching to a different photo that has a background/foreground

image_path = 'images\DSC08708.jpg'
with open(image_path, 'rb') as f:
    image_data = f.read()

Image.open(image_path)

In [ ]:
api_version = "2023-02-01-preview"
mode="backgroundRemoval"
url = f"{endpoint}computervision/imageanalysis:segment?api-version={api_version}&mode={mode}"

headers= {
    "Ocp-Apim-Subscription-Key": api_key, 
    "Content-Type": "application/octet-stream" 
}
    
response = requests.post(url, headers=headers, data=image_data)

modified_image = response.content

foreground_path = "images/backgroundForeground.png"
with open(foreground_path, "wb") as file:
    file.write(modified_image)

Image.open(foreground_path)

In [ ]:
api_version = "2023-02-01-preview"
mode="foregroundMatting"
url = f"{endpoint}computervision/imageanalysis:segment?api-version={api_version}&mode={mode}"

headers= {
    "Ocp-Apim-Subscription-Key": api_key, 
    "Content-Type": "application/octet-stream" 
}
    
response = requests.post(url, headers=headers, data=image_data)

modified_image = response.content

foreground_path = "images/backgroundForeground.png"
with open(foreground_path, "wb") as file:
    file.write(modified_image)

Image.open(foreground_path)